# Duck Segmentation

This project builds an ML model that performs segmentation on the images that contain ducks.

The architecture used for segmentatic segmention is YOLO (YOLOv8).

---

## Image Labelling

This image labelling tool used is: CVAT.

For segmentation, the annotator used is primarily polygons, with coordinates available to download.

<img src='snapshots/image-annotation.png' width='70%' height='auto' />

With this, the **binary mask** is created for the respected image.

<img src='label_samples/masks/00eb7860432c771a.png' width='50%' height='auto' />

However, **label coordinates** are required for **YOLOv8**.

### From binary mask to label coordinates

In [19]:
import os
import cv2

mask_dir = os.path.join('.', 'label_samples', 'masks')
label_dir = os.path.join('.', 'label_samples', 'labels')

for file in os.listdir(mask_dir):
    mask_path = os.path.join(mask_dir, file)
    
    # Load binary mask
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
    _, mask = cv2.threshold(mask, 1, 255, cv2.THRESH_BINARY)
    
    mask_h, mask_w = mask.shape
    contours, hierarchy = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Converting contours to coordinates (polygons)
    coordinates = []
    for cnt in contours:
        if cv2.contourArea(cnt) > 200:
            coordinate = []
            for point in cnt:
                x, y = point[0]
                coordinate.append(x / mask_w)
                coordinate.append(y / mask_h)
            coordinates.append(coordinate)
            
    # Printing coordinates
    with open('{}.txt'.format(os.path.join(label_dir, file)[:-4]), 'w') as f:
        for coordinate in coordinates:
            for p_, p in enumerate(coordinate):
                if p_ == len(coordinate) - 1:
                    f.write('{}\n'.format(p))
                elif p_ == 0:
                    f.write('0 {} '.format(p))
                else:
                    f.write('{} '.format(p))

        f.close()

---

## Data preprocessing

The data structure for this project is:
* data/
    * images/
        * train/
        * val/
    * labels/
        * train/
        * val/

---

## Loading a pretrained model

Performing transfer learning.

In [7]:
%%writefile config.yaml
path: D:\Personal\Projects\images\computer-vision\duck-segmentation # data path
train: images/train
val: images/val

nc: 1 # number of classes
names: ['duck'] # lass label

Overwriting config.yaml


In [14]:
from ultralytics import YOLO

model = YOLO('yolov8n-seg.pt') # Loading pretrained model

model.train(data='config.yaml', epochs=10, imgsz=640, batch=16) # Training pretrained model

New https://pypi.org/project/ultralytics/8.0.212 available  Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.112  Python-3.11.3 torch-2.0.1+cu117 CUDA:0 (NVIDIA GeForce MX250, 2048MiB)
yolo\engine\trainer: task=segment, mode=train, model=yolov8n-seg.pt, data=config.yaml, epochs=10, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=False, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=0, resume=False, amp=True, fraction=1.0, profile=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, 

OutOfMemoryError: CUDA out of memory. Tried to allocate 2.00 MiB (GPU 0; 2.00 GiB total capacity; 1.09 GiB already allocated; 0 bytes free; 1.12 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [15]:
!start .